# SN — Corporate Rating Model

This notebook implements and documents a corporate credit rating model that combines quantitative financial metrics with qualitative assessments.

## Structure overview

**SN classes**
1. `SN_RatingComponents` – structured container for all key outputs (scores, ratings, outlook).
2. `SN_CorporateRatingModel` – rating engine that computes scores, applies caps, and produces final ratings.

**Top-level functions**
- `level_score()` – map ratios → level → score (critical).
- `score_to_rating()` – map numeric score → rating (critical).
- `apply_sovereign_cap()` – enforce sovereign ceiling on issuer rating (important).
- `rating_band()` – return rating band range for outlook logic (important).

**Model methods (SN_CorporateRatingModel)**
- `__init__` – configure and normalize quantitative/qualitative weights (important).
- `compute_quantitative_score()` – compute quantitative block using financial ratios (critical).
- `compute_qualitative_score()` – compute qualitative block from 1–5 factor inputs (critical).
- `combine_scores()` – apply weights to quantitative and qualitative scores (critical).
- `derive_outlook()` – derive Positive / Stable / Negative from score within rating band (important).
- `rate()` – orchestrate the full rating process and return `SN_RatingComponents` (critical).

**Execution flow (simplified)**

`rate()`  
├── `compute_quantitative_score()` → `level_score()`  
├── `compute_qualitative_score()`  
├── `combine_scores()`  
├── `score_to_rating()`  
├── `apply_sovereign_cap()`  
└── `derive_outlook()`


**USER INPUT SECTION**
- Change the marked "input values" to run your own scenario

In [118]:
# ================== USER INPUT SECTION ==================
# Adjust ONLY this section when running the model

# Weights for quantitative vs qualitative blocks
W_QUANTITATIVE = 0.65 # input values
W_QUALITATIVE = 0.35 # input values

# Financial ratios (replace with your inputs)
# IMPORTANT: ratios like margins, ROE, ROA, ROIC, debt_to_capital are decimals (e.g. 0.20 = 20%)
FINANCIALS_INPUT = {
    "debt_ebitda": 3.0, # input values
    "ffo_debt": 0.25, # input values
    "ebitda_interest": 4.0, # input values
    "ffo_interest": 3.5, # input values
    "cfo_interest": 3.0, # input values
    "cfo_debt": 0.20, # input values
    "debt_to_capital": 0.45,  # input values     
    "ebit_margin": 0.12, # input values
    "ebitda_margin": 0.20, # input values
    "roe": 0.14, # input values
    "roa": 0.05, # input values
    "roic": 0.11, # input values
    "current_ratio": 1.4, # input values
    "quick_ratio": 1.1, # input values
    "cash_st_debt": 0.8, # input values
    "cash_total_debt": 0.25, # input values
    "maturity_wall_ratio": 1.2, # input values
}

# Qualitative scores (1–5):
# 1 = very strong, 3 = mid-point/neutral, 5 = very weak.
QUALITATIVE_INPUT = {
    "business_risk": {
        "industry_risk": 3, # input values
        "market_position": 3, # input values          # 3 = neutral / average risk
        "revenue_stability": 3, # input values
    },
    "management_governance": {
        "management_quality": 3, # input values
        "governance": 3, # input values
        "financial_policy": 3, # input values
    },
    "country_structural": {
        "sovereign_risk": 3, # input values
        "legal_environment": 3, # input values
    },
}

# Sovereign information (optional)
SOVEREIGN_RATING = "BBB" # input values        # e.g. "BBB", "AAA", or None
SOVEREIGN_OUTLOOK = "Negative" # input values  # "Positive", "Stable", "Negative", or None


In [119]:
# ============== LOGGING SETUP ==============
import logging
from dataclasses import dataclass
from statistics import mean

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)



In [120]:
# ============== RATING GRID AND ORDER (MIN, MAX, RATING) ==============

RATING_GRID = [
    (95, 100, "AAA"),
    (90, 94, "AA+"),
    (85, 89, "AA"),
    (80, 84, "AA-"),
    (75, 79, "A+"),
    (70, 74, "A"),
    (65, 69, "A-"),
    (60, 64, "BBB+"),
    (55, 59, "BBB"),
    (50, 54, "BBB-"),
    (45, 49, "BB+"),
    (40, 44, "BB"),
    (35, 39, "BB-"),
    (30, 34, "B+"),
    (25, 29, "B"),
    (20, 24, "B-"),
    (15, 19, "CCC+"),
    (10, 14, "CCC"),
    (0,  9,  "CCC-"),
]

RATING_ORDER = [
    "AAA", "AA+", "AA", "AA-",
    "A+", "A", "A-",
    "BBB+", "BBB", "BBB-",
    "BB+", "BB", "BB-",
    "B+", "B", "B-",
    "CCC+", "CCC", "CCC-",
]


RATING_TO_INDEX = {r: i for i, r in enumerate(RATING_ORDER)} # Assigns the rating order with the index like AAA:0, AA+:1 ...

## Ratio thresholds (1–5 levels)
The next cell defines all 1–5 threshold grids for the financial ratios.
You can skip it on first read and come back when you need the exact cut-offs.


In [122]:
# ============== 1–5 LEVEL GRIDS FOR RATIOS ==============
# Level 1 = strongest, Level 5 = weakest

# Leverage / coverage

DEBT_EBITDA_LEVELS = [
    (float("-inf"), 1.0, 1),
    (1.0, 2.0, 2),
    (2.0, 3.5, 3),
    (3.5, 5.0, 4),
    (5.0, float("inf"), 5),
]

EBITDA_INT_LEVELS = [
    (8.0, float("inf"), 1),
    (4.0, 8.0, 2),
    (2.5, 4.0, 3),
    (1.5, 2.5, 4),
    (float("-inf"), 1.5, 5),
]

CFO_INT_LEVELS = [
    (7.0, float("inf"), 1),
    (4.0, 7.0, 2),
    (2.5, 4.0, 3),
    (1.5, 2.5, 4),
    (float("-inf"), 1.5, 5),
]

CFO_DEBT_LEVELS = [
    (0.35, float("inf"), 1),
    (0.25, 0.35, 2),
    (0.15, 0.25, 3),
    (0.08, 0.15, 4),
    (float("-inf"), 0.08, 5),
]

# Debt to Capital (Debt / (Debt + Equity)), lower is better

DEBT_TO_CAPITAL_LEVELS = [
    (float("-inf"), 0.20, 1),   # < 20%
    (0.20, 0.35, 2),            # 20–35%
    (0.35, 0.50, 3),            # 35–50%
    (0.50, 0.65, 4),            # 50–65%
    (0.65, float("inf"), 5),    # > 65%
]

# Profitability / returns (ratios as decimals)

EBIT_MARGIN_LEVELS = [
    (0.20, float("inf"), 1),
    (0.12, 0.20, 2),
    (0.08, 0.12, 3),
    (0.04, 0.08, 4),
    (float("-inf"), 0.04, 5),
]

EBITDA_MARGIN_LEVELS = [
    (0.30, float("inf"), 1),
    (0.20, 0.30, 2),
    (0.12, 0.20, 3),
    (0.08, 0.12, 4),
    (float("-inf"), 0.08, 5),
]

ROE_LEVELS = [
    (0.20, float("inf"), 1),
    (0.12, 0.20, 2),
    (0.08, 0.12, 3),
    (0.04, 0.08, 4),
    (float("-inf"), 0.04, 5),
]

ROA_LEVELS = [
    (0.08, float("inf"), 1),
    (0.05, 0.08, 2),
    (0.03, 0.05, 3),
    (0.015, 0.03, 4),
    (float("-inf"), 0.015, 5),
]

# Liquidity / maturity

CURRENT_RATIO_LEVELS = [
    (2.0, float("inf"), 1),
    (1.5, 2.0, 2),
    (1.2, 1.5, 3),
    (1.0, 1.2, 4),
    (float("-inf"), 1.0, 5),
]

CASH_ST_DEBT_LEVELS = [
    (1.0, float("inf"), 1),
    (0.7, 1.0, 2),
    (0.4, 0.7, 3),
    (0.2, 0.4, 4),
    (float("-inf"), 0.2, 5),
]

CASH_TOTAL_DEBT_LEVELS = [
    (0.40, float("inf"), 1),
    (0.25, 0.40, 2),
    (0.15, 0.25, 3),
    (0.08, 0.15, 4),
    (float("-inf"), 0.08, 5),
]

MATURITY_WALL_LEVELS = [
    (float("-inf"), 0.5, 1),
    (0.5, 1.0, 2),
    (1.0, 1.8, 3),
    (1.8, 3.0, 4),
    (3.0, float("inf"), 5),
]


# Explicit 1–5 -> 0–100 mapping
LEVEL_TO_SCORE = {
    1: 100.0,
    2: 75.0,
    3: 50.0,
    4: 25.0,
    5: 0.0,
}


In [123]:
# ================== DATA STRUCTURE ==================

from dataclasses import dataclass

@dataclass
class SN_RatingComponents:
    """
    SN_RatingComponents

    Purpose:
    ----------
    Acts as the structured output container for the corporate rating model.
    It bundles all key results of the rating process into a single, consistent object.

    What it contains:
    ------------------
    - quantitative_score : Final 0–100 score derived from financial ratios
    - qualitative_score  : Final 0–100 score derived from qualitative factors
    - combined_score     : Weighted average of quantitative and qualitative scores
    - uncapped_rating    : Rating implied by the combined score before sovereign constraints
    - final_rating       : Rating after applying sovereign cap (if applicable)
    - outlook            : Rating outlook (Positive / Stable / Negative)
    - capped_by_sovereign: Indicates whether the sovereign rating limited the issuer rating

    Why it is important:
    ---------------------
    - Provides a single, well-defined “result object” for the model
    - Prevents fragile tuple-based or dictionary-based returns
    - Makes outputs easy to log, report, serialize, or pass to APIs
    - Improves readability, maintainability, and future extensibility
    """
    quantitative_score: float
    qualitative_score: float
    combined_score: float
    uncapped_rating: str
    final_rating: str
    outlook: str
    capped_by_sovereign: bool


In [124]:
# The function returns a rating to a score (#) as per the Rating grid
def score_to_rating(score: float) -> str:
    for min_s, max_s, rating in RATING_GRID:
        if min_s <= score <= max_s:
            return rating
    return "CCC-"

In [125]:
# The function returns the min and max score band for a given rating as per the Rating grid

def rating_band(rating: str) -> tuple[float, float] | None:
    for min_s, max_s, r in RATING_GRID:
        if r == rating:
            return (min_s, max_s)
    return None

In [126]:
# If the issuer rating is better than the sovereign rating, downgrade it to (i.e. cap it at) the sovereign level.
def apply_sovereign_cap(corp_rating: str, sovereign_rating: str) -> str:
    if corp_rating not in RATING_TO_INDEX or sovereign_rating not in RATING_TO_INDEX:
        return corp_rating
    corp_idx = RATING_TO_INDEX[corp_rating]
    sov_idx = RATING_TO_INDEX[sovereign_rating]
    capped_idx = max(corp_idx, sov_idx)
    return RATING_ORDER[capped_idx]



In [127]:
#  This function converts a numeric input value of the ratios into a score in line with the LEVEL_TO_SCORE. 
#  For eg., a value of 2 for a debt_ebitda will be mapped as level 3 per DEBT_EBITDA_LEVELS and then to a score of 50  per LEVEL_TO_SCORE
# thereby all the quantitative ratios get a score to be used for the rating calibration.

def level_score(
    value: float,
    levels: list[tuple[float, float, int]],
    *,
    force_negative_worst: bool = False,
    ratio_name: str = "",
) -> float | None:
    """
    Map a raw ratio value to a 1–5 level, then to a 0–100 score
    using LEVEL_TO_SCORE (1->100, 2->75, 3->50, 4->25, 5->0).

    This function is generic and reused for all quantitative ratios by passing
    the appropriate level grid (e.g. DEBT_EBITDA_LEVELS, FFO_DEBT_LEVELS).
    """
    if value is None:
        return None

    if force_negative_worst and value < 0:
        level = 5
        score = LEVEL_TO_SCORE[level]
        logging.info(
            "Quant ratio %s: value=%.2f is negative -> forced worst level=%d, score=%.1f",
            ratio_name,
            value,
            level,
            score,
        )
        return score

    for lower, upper, level in levels:
        if lower <= value < upper:
            score = LEVEL_TO_SCORE.get(level, 0.0)
            logging.info(
                "Quant ratio %s: value=%.2f -> level=%d -> score=%.1f",
                ratio_name,
                value,
                level,
                score,
            )
            return score

    return None


In [128]:
# ================== MODEL ==================

class SN_CorporateRatingModel:
    """
    SN_CorporateRatingModel

    Purpose:
    ----------
    Implements a structured corporate credit rating engine that combines quantitative
    financial analysis with qualitative business and governance assessment.
    It maps raw inputs (financial ratios and qualitative scores) into a final rating
    and outlook, optionally constrained by a sovereign cap.

    What it does:
    ---------------
    - Computes a quantitative score from a set of financial ratios (leverage, profitability,
      liquidity, maturity structure) using predefined level mappings.
    - Computes a qualitative score from business risk, management/governance, and
      country/structural factors, interpreted on a 1–5 scale.
    - Combines the two scores using configurable weights (quantitative vs qualitative).
    - Maps the combined score to a rating band and derives an outlook (Positive / Stable / Negative).
    - Applies a sovereign cap when a sovereign rating is provided, ensuring the issuer
      rating does not exceed the sovereign.
    - Returns a structured SN_RatingComponents object containing all intermediate and final outputs.

    Key methods:
    -------------
    - __init__                  : Initializes the model with normalized quantitative/qualitative weights.
    - compute_quantitative_score: Computes the quantitative score from financial ratios.
    - compute_qualitative_score : Computes the qualitative score from qualitative inputs.
    - combine_scores            : Combines quantitative and qualitative scores using weights.
    - derive_outlook            : Derives outlook from score within the rating band.
    - rate                      : Main entry point that orchestrates the full rating process.

    Why it is important:
    ---------------------
    - Encapsulates the full rating logic in a single, reusable class.
    - Separates concerns: quantitative block, qualitative block, combination, cap, and outlook.
    - Makes the model configurable (via weights) and transparent (via logging and scores).
    - Provides a clear interface (rate(...)) and a well-defined output type (SN_RatingComponents).
    """

    def __init__(
        self,
        w_quantitative: float = 0.65,
        w_qualitative: float = 0.35,
    ):
        # Normalize the user‑provided weights so they sum to 1.0.
        # This ensures the quantitative and qualitative components
        # are always treated as proportions of the total score.        
        total_weight = w_quantitative + w_qualitative
        self.w_quantitative = w_quantitative / total_weight
        self.w_qualitative = w_qualitative / total_weight

    # ---------- QUANTITATIVE BLOCK ----------

    def compute_quantitative_score(self, financials: dict) -> float:
        # Compute an overall quantitative score by averaging scores from a set of financial ratios (leverage, profitability, liquidity, etc.).
        scores: list[float] = []

        def add_ratio(
            name: str,
            key: str,
            levels: list[tuple[float, float, int]],
            force_negative_worst: bool = False,
        ):
        # Helper function that:
        # - looks up a ratio value in the financials dict,
        # - maps it to a score using level_score,
        # - optionally treats negative values as “worst” case,
        # - appends the score to the scores list if valid.
            if key in financials:
                value = financials[key]
                score = level_score(
                    value,
                    levels,
                    force_negative_worst=force_negative_worst,
                    ratio_name=name,
                )
                if score is not None:
                    scores.append(score)

        # Leverage / coverage
        add_ratio("debt_ebitda", "debt_ebitda", DEBT_EBITDA_LEVELS, force_negative_worst=True)
        add_ratio("ebitda_interest", "ebitda_interest", EBITDA_INT_LEVELS, force_negative_worst=True)
        add_ratio("cfo_interest", "cfo_interest", CFO_INT_LEVELS, force_negative_worst=True)
        add_ratio("cfo_debt", "cfo_debt", CFO_DEBT_LEVELS, force_negative_worst=True)
        add_ratio("debt_to_capital", "debt_to_capital", DEBT_TO_CAPITAL_LEVELS, force_negative_worst=True)

        # Profitability / returns
        add_ratio("ebit_margin", "ebit_margin", EBIT_MARGIN_LEVELS, force_negative_worst=True)
        add_ratio("ebitda_margin", "ebitda_margin", EBITDA_MARGIN_LEVELS, force_negative_worst=True)
        add_ratio("roe", "roe", ROE_LEVELS, force_negative_worst=True)
        add_ratio("roa", "roa", ROA_LEVELS, force_negative_worst=True)

        # Liquidity / maturity
        add_ratio("current_ratio", "current_ratio", CURRENT_RATIO_LEVELS)
        add_ratio("cash_st_debt", "cash_st_debt", CASH_ST_DEBT_LEVELS)
        add_ratio("cash_total_debt", "cash_total_debt", CASH_TOTAL_DEBT_LEVELS)
        add_ratio("maturity_wall_ratio", "maturity_wall_ratio", MATURITY_WALL_LEVELS)

        if not scores:
        # If no valid ratios are provided, default the quantitative score to 50.0.
            logging.warning("No quantitative ratios provided; defaulting quantitative_score=50.0")
            return 50.0
            
        # Take the mean of all ratio scores and round to one decimal place.
        quantitative_score = round(mean(scores), 1)
        logging.info("Quantitative block score (avg of ratios) = %.1f", quantitative_score)
        return quantitative_score

    # ---------- QUALITATIVE BLOCK ----------

    def compute_qualitative_score(self, qualitative_inputs: dict) -> float:
        # Compute an overall qualitative score by averaging scores from business risk, management/governance, and country/structural factors.
        all_scores: list[float] = []

        for bucket_name in ["business_risk", "management_governance", "country_structural"]:
            factors = qualitative_inputs.get(bucket_name, {})
            for factor_name, value in factors.items():
                if value is None:
                    continue

                try:
                    v = float(value)
                except (TypeError, ValueError):
                # Skip non‑numeric qualitative inputs and log a warning.
                    logging.warning(
                        "Qual factor %s.%s: non-numeric value=%r -> skipped",
                        bucket_name,
                        factor_name,
                        value,
                    )
                    continue

                if not (1.0 <= v <= 5.0):
                # Skip values outside the allowed 1–5 scale and log a warning.
                    logging.warning(
                        "Qual factor %s.%s: value %.2f outside [1.0, 5.0] -> skipped",
                        bucket_name,
                        factor_name,
                        v,
                    )
                    continue

                # Interpret 1 = best, 5 = worst, and use same LEVEL_TO_SCORE
                level = int(round(v))
                score = LEVEL_TO_SCORE.get(level, 50.0)
                all_scores.append(score)

                logging.info(
                    "Qual factor %s.%s: value=%r -> level=%d -> score=%.1f",
                    bucket_name,
                    factor_name,
                    value,
                    level,
                    score,
                )

        if not all_scores:
        # If no valid qualitative inputs, default the qualitative score to 50.0.
            logging.warning("No valid qualitative inputs; defaulting qualitative_score=50.0")
        return 50.0

    # Take the mean of all qualitative factor scores and round to one decimal place.
        qualitative_score = round(mean(all_scores), 1)
        logging.info("Qualitative block score (avg of factors) = %.1f", qualitative_score)
        return qualitative_score

    # ---------- COMBINATION, RATING & OUTLOOK ----------

    def combine_scores(self, quantitative: float, qualitative: float) -> float:
        # Combine the quantitative and qualitative scores using the normalized weights stored in self.w_quantitative and self.w_qualitative.
        combined = (
            self.w_quantitative * quantitative
            + self.w_qualitative * qualitative
        )
        combined_score = round(combined, 1)
        logging.info(
            "Combined score = %.1f (quantitative=%.1f, qualitative=%.1f)",
            combined_score,
            quantitative,
            qualitative,
        )
        return combined_score

    def derive_outlook(self, combined_score: float, rating: str) -> str:
        # Derive an outlook (Positive / Stable / Negative) based on where the combined score sits within the rating band of the given rating.
        band = rating_band(rating)
        if band is None:
            logging.warning(
                "derive_outlook: rating %s not found in RATING_GRID -> default Stable",
                rating,
            )
            return "Stable"

        lower_edge, upper_edge = band

        logging.info(
            "Outlook band for %s: lower_edge=%.1f, upper_edge=%.1f, score=%.1f",
            rating,
            lower_edge,
            upper_edge,
            combined_score,
        )

        if combined_score == upper_edge:
            return "Positive"
        if combined_score == lower_edge:
            return "Negative"
        return "Stable"

    def rate(
        self,
        financials: dict,
        qualitative_inputs: dict,
        sovereign_rating: str | None = None,
        sovereign_outlook: str | None = None,
     # Main rating engine method:
        # 1) compute quantitative and qualitative scores,
        # 2) combine them into a single score,
        # 3) map that score to a rating,
        # 4) apply a sovereign cap if a sovereign rating is provided,
        # 5) derive or anchor the outlook,
        # 6) return a structured result object (SN_RatingComponents).
    ) -> SN_RatingComponents:
        quantitative_score = self.compute_quantitative_score(financials)
        qualitative_score = self.compute_qualitative_score(qualitative_inputs)

        combined_score = self.combine_scores(quantitative_score, qualitative_score)
        uncapped_rating = score_to_rating(combined_score)
        logging.info(
            "Uncapped rating from combined_score=%.1f -> %s",
            combined_score,
            uncapped_rating,
        )

        capped = False
        if sovereign_rating:
        # Apply a sovereign cap: issuer rating cannot be better than sovereign.
            final_rating = apply_sovereign_cap(uncapped_rating, sovereign_rating)
            capped = (final_rating != uncapped_rating)
            logging.info(
                "Applying sovereign cap: sovereign=%s, uncapped=%s, final=%s, capped=%s",
                sovereign_rating,
                uncapped_rating,
                final_rating,
                capped,
            )
        else:
            final_rating = uncapped_rating

        if capped and sovereign_outlook:
        # If sovereign cap is binding, anchor the issuer outlook to the sovereign outlook.
            outlook = sovereign_outlook
            logging.info(
                "Outlook anchored to sovereign (cap binding): sovereign_outlook=%s -> issuer_outlook=%s",
                sovereign_outlook,
                outlook,
            )
        elif capped and not sovereign_outlook:
        # If cap is binding but no sovereign outlook, default to Stable.
            outlook = "Stable"
            logging.info(
                "Outlook defaulted to Stable (cap binding, no sovereign_outlook provided)"
            )
        else:
        # Otherwise derive the outlook from the issuer’s rating band and combined score.
            outlook = self.derive_outlook(combined_score, uncapped_rating)
            logging.info("Outlook derived from issuer band (no binding sovereign cap): %s",outlook,
            )

        # AAA is the highest rating, so we never show a Positive outlook (no upgrade beyond AAA).
        # If derive_outlook() suggests Positive, override it to Stable.
        if final_rating == "AAA" and outlook == "Positive":
            logging.info(
                "Final rating is AAA and outlook was Positive -> forcing outlook to Stable"
            )
            outlook = "Stable"

        # Package all intermediate and final outputs into a single result object.
        return SN_RatingComponents(
            quantitative_score=quantitative_score,
            qualitative_score=qualitative_score,
            combined_score=combined_score,
            uncapped_rating=uncapped_rating,
            final_rating=final_rating,
            outlook=outlook,
            capped_by_sovereign=capped,
        )



In [129]:
# ================== SINGLE SAMPLE RUN ==================
# Run one example issuer through the model and print the results.

# Create an instance of the rating model with user-defined weights.
# W_QUANTITATIVE and W_QUALITATIVE control how much weight goes to financials vs qualitative factors.
model = SN_CorporateRatingModel(
    w_quantitative=W_QUANTITATIVE,
    w_qualitative=W_QUALITATIVE,
)

# Run the full rating engine on the input data:
# - compute quantitative score from FINANCIALS_INPUT,
# - compute qualitative score from QUALITATIVE_INPUT,
# - combine scores using the weights,
# - derive an uncapped rating,
# - apply sovereign cap if SOVEREIGN_RATING is provided,
# - determine outlook (possibly anchored to SOVEREIGN_OUTLOOK),
# - return a structured result object (SN_RatingComponents).
result = model.rate(
    FINANCIALS_INPUT,
    QUALITATIVE_INPUT,
    sovereign_rating=SOVEREIGN_RATING,
    sovereign_outlook=SOVEREIGN_OUTLOOK,
)

# Create a short text label indicating whether the final rating was capped by the sovereign rating.
cap_text = "(capped by sovereign)" if result.capped_by_sovereign else "(not capped by sovereign)"

# Print a simple summary of the outcome.
print("Quantitative score:", result.quantitative_score)
print("Qualitative score:", result.qualitative_score)
print("Combined score:", result.combined_score)
print(f"Uncapped rating: {result.uncapped_rating}")
print(f"Final rating: {result.final_rating} {cap_text}")
print("Outlook:", result.outlook)


2026-02-05 12:30:45,176 - INFO - Quant ratio debt_ebitda: value=3.00 -> level=3 -> score=50.0
2026-02-05 12:30:45,179 - INFO - Quant ratio ebitda_interest: value=4.00 -> level=2 -> score=75.0
2026-02-05 12:30:45,181 - INFO - Quant ratio cfo_interest: value=3.00 -> level=3 -> score=50.0
2026-02-05 12:30:45,182 - INFO - Quant ratio cfo_debt: value=0.20 -> level=3 -> score=50.0
2026-02-05 12:30:45,183 - INFO - Quant ratio debt_to_capital: value=0.45 -> level=3 -> score=50.0
2026-02-05 12:30:45,184 - INFO - Quant ratio ebit_margin: value=0.12 -> level=2 -> score=75.0
2026-02-05 12:30:45,184 - INFO - Quant ratio ebitda_margin: value=0.20 -> level=2 -> score=75.0
2026-02-05 12:30:45,185 - INFO - Quant ratio roe: value=0.14 -> level=2 -> score=75.0
2026-02-05 12:30:45,185 - INFO - Quant ratio roa: value=0.05 -> level=2 -> score=75.0
2026-02-05 12:30:45,185 - INFO - Quant ratio current_ratio: value=1.40 -> level=3 -> score=50.0
2026-02-05 12:30:45,185 - INFO - Quant ratio cash_st_debt: value=0

Quantitative score: 61.8
Qualitative score: 50.0
Combined score: 57.7
Uncapped rating: BBB
Final rating: BBB (not capped by sovereign)
Outlook: Stable
